### Data preparations

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install lightning
!pip install pytorchvideo

In [3]:
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler, labeled_video_dataset
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    UniformTemporalSubsample,
    Permute
)
from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize
)
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo
)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [4]:
video_transforms = Compose([
    ApplyTransformToKey(key='video',
    transform=Compose([
        UniformTemporalSubsample(20),
        Normalize((0.45, 0.45, 0.45),(0.225, 0.225, 0.225)),
        # RandomShortSideScale(min_size=248, max_size=256),
        # CenterCropVideo(224),
        RandomHorizontalFlip(p=0.5),
    ]),
    ),
])

In [5]:
import pandas as pd
import numpy as np
import os
import shutil

In [6]:
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor

In [7]:
dataset_path = "/content/drive/MyDrive/cells_dataset/dataset"

In [8]:
os.chdir(dataset_path)
os.chdir("..")

In [9]:
dataframe = pd.read_csv("DataFrame.csv", index_col=0)
dataframe = dataframe.sample(frac=1)
ratio = 0.8
train_size = int(dataframe.shape[0] * ratio)
train_data = dataframe[0:train_size]
test_data = dataframe[train_size:]
train_data, test_data

(            files  erythrocytes  lymphocytes  high_erythrocytes  \
 593    traffic594          4942           10                  0   
 1147  traffic1148          4918          276                  0   
 389    traffic390          4966          219                  0   
 18      traffic19          4904          229                  0   
 55      traffic56          5198          301                  1   
 ...           ...           ...          ...                ...   
 785    traffic786          4915           96                  0   
 1082  traffic1083          5036          144                  1   
 905    traffic906          4961          208                  0   
 961    traffic962          5101          157                  1   
 649    traffic650          4855          281                  0   
 
       high_lymphocytes  blur  noise  
 593                  0   0.0      1  
 1147                 1  10.0      1  
 389                  1   4.0      0  
 18                   1   

In [10]:
class CustomDataset(LabeledVideoDataset):
    def __init__(self, dataset_path, dataframe, target_name, transforms, clip_sampler_type='random', clip_duration=1):
      df = dataframe.reset_index()
      paths = []
      for i, file_name in enumerate(df['files']):
          temp_dict = df.iloc[i].to_dict()
          temp_dict['label'] = df[target_name][i]
          temp_dict.pop('files')
          temp_dict.pop('index')
          paths.append((f"{dataset_path}/{file_name}", temp_dict))
      super().__init__(labeled_video_paths=paths,
                       clip_sampler=make_clip_sampler(clip_sampler_type, clip_duration),
                       transform=transforms, decode_audio=False)

In [11]:
train_dataset = CustomDataset(dataset_path=dataset_path, dataframe=train_data,
                              target_name='high_erythrocytes',
                              transforms=video_transforms)

In [12]:
train_dataset.num_videos

920

In [11]:
checkpoint_callback = ModelCheckpoint(monitor="val_loss", dirpath="checkpoints", filename="file", save_last=True)
lr_monitor = LearningRateMonitor(logging_interval="epoch")

In [12]:
class CurriculumTrainer():
    def set_difficulty(self, dataframe, target_cells, target_name, alpha, beta, gamma):
        # Calculating distances between mean and each target value
        dataframe['distances'] = [dataframe[target_cells].mean() - value for value in dataframe[target_cells]]

        # Normalizing blur and distances
        #df['blur'] = (df['blur'] - df['blur'].mean()) / df['blur'].std()
        dataframe['distances'] = (dataframe['distances'] - dataframe['distances'].mean()) / dataframe['distances'].std()

        # Calculating and normalizing difficulty
        dataframe['difficulty'] = dataframe['blur'] * alpha + dataframe['noise'] * beta + dataframe['distances'] * gamma
        dataframe['difficulty'] = (dataframe['difficulty'] - dataframe['difficulty'].min()) / (dataframe['difficulty'].max() - dataframe['difficulty'].min())
        return dataframe

    def evaluate_competence(self, max_epochs, current_epoch, c0, p):
        return min(1, (current_epoch*((1-c0**p)/max_epochs)+c0**p)**(1/p))

    def fit(self, model, dataframe, target_cells, target_name, max_epochs, c0, p, alpha, beta, gamma):
        dataframe = self.set_difficulty(dataframe, target_cells, target_name, alpha, beta, gamma)
        self.competence = c0
        seed_everything(0)
        for epoch in range(max_epochs):
            print(f"-----------------------\nEpoch {epoch}, competence = {self.competence}")
            selected_data = dataframe[dataframe.difficulty < self.competence]
            dataset = CustomDataset(dataset_path=dataset_path, dataframe=selected_data,
                                    target_name=target_name,
                                    transforms=video_transforms)
            loader = DataLoader(dataset, batch_size=model.batch_size, num_workers=model.numworkers, pin_memory=True)
            self.trainer = Trainer(max_epochs=1,
                                  precision=16,
                                  accumulate_grad_batches=2,
                                  enable_progress_bar=False,
                                  enable_model_summary=False,
                                  num_sanity_val_steps=0,
                                  callbacks=[lr_monitor, checkpoint_callback])
            self.trainer.fit(model, loader)
            self.competence = self.evaluate_competence(max_epochs, epoch, c0, p)

    def validate(self, model):
        dataset = CustomDataset(dataset_path=dataset_path, dataframe=test_data,
                                target_name=model.target_label,
                                transforms=video_transforms)
        loader = DataLoader(dataset, batch_size=model.batch_size, num_workers=model.numworkers, pin_memory=True)
        self.trainer.validate(model, loader)

    def test(self, model):
        dataset = CustomDataset(dataset_path=dataset_path, dataframe=test_data,
                                target_name=model.target_label,
                                transforms=video_transforms)
        loader = DataLoader(dataset, batch_size=model.batch_size, num_workers=model.numworkers, pin_memory=True)
        self.trainer.validate(model, loader)

### Model

In [13]:
import torch
import torch.nn as nn
from pytorch_lightning import LightningModule, seed_everything, Trainer
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import classification_report
import torchmetrics

In [14]:
class TestModel(LightningModule):
    def __init__(self, target_label, num_classes=2):
        super(TestModel, self).__init__()
        # model architecture
        self.video_model = torch.hub.load("facebookresearch/pytorchvideo", "efficient_x3d_xs", pretrained=True)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(400, num_classes)

        self.lr = 1e-3
        self.batch_size = 8
        self.numworkers = 0
        # evaluation metric
        self.metric = torchmetrics.Accuracy(task='multiclass', num_classes=num_classes)
        # loss function
        self.criterion = nn.CrossEntropyLoss()
        # helpers
        self.target_label = target_label
        self.training_step_outputs = []
        self.validation_step_outputs = []
        self.testing_step_outputs = []

    def forward(self, x):
        x = self.video_model(x)
        x = self.relu(x)
        x = self.linear(x)
        return x

    def configure_optimizers(self):
        opt = torch.optim.AdamW(params=self.parameters(), lr=self.lr)
        scheduler = CosineAnnealingLR(opt, T_max=10, eta_min=1e-6, last_epoch=-1)
        return {'optimizer': opt, 'lr_scheduler': scheduler}

    # def train_dataloader(self):
    #     dataset = CustomDataset(dataset_path=dataset_path, dataframe=train_data,
    #                           target_name=self.target_label,
    #                           transforms=video_transforms)
    #     loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworkers, pin_memory=True)
    #     return loader

    def training_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self.forward(video)
        loss = self.criterion(out, label)
        metric = self.metric(out, label.to(torch.int64))
        self.training_step_outputs.append({'loss': loss, 'metric': metric})
        print({'loss': loss, 'metric': metric})
        return {'loss': loss, 'metric': metric}

    def on_train_epoch_end(self):
        outputs = self.training_step_outputs
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        self.training_step_outputs = []
        self.log('train_loss', loss)
        self.log('train_metric', metric)

    # def val_dataloader(self):
    #     dataset = CustomDataset(dataset_path=dataset_path, dataframe=test_data,
    #                           target_name=self.target_label,
    #                           transforms=video_transforms)
    #     loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworkers, pin_memory=True)
    #     return loader

    def validation_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self.forward(video)
        loss = self.criterion(out, label)
        metric = self.metric(out, label.to(torch.int64))
        self.validation_step_outputs.append({'loss': loss, 'metric': metric})
        return {'loss': loss, 'metric': metric}

    def on_validation_epoch_end(self):
        outputs = self.validation_step_outputs
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        self.validation_step_outputs = []
        self.log('val_loss', loss)
        self.log('val_metric', metric)

    # def test_dataloader(self):
    #     dataset = CustomDataset(dataset_path=dataset_path, dataframe=test_data,
    #                           target_name=self.target_label,
    #                           transforms=video_transforms)
    #     loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworkers, pin_memory=True)
    #     return loader

    def test_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self.forward(video)
        self.testing_step_outputs.append({'label': label, 'pred': out})
        return {'label': label, 'pred': out}

    def on_test_epoch_end(self):
        outputs = self.testing_step_outputs
        label = torch.cat([x['label'] for x in outputs]).cpu().detach().numpy()
        pred = torch.cat([x['pred'].argmax(dim=1) for x in outputs]).cpu().detach().numpy()
        self.testing_step_outputs = []
        print(classification_report(label, pred))

In [15]:
model = TestModel(num_classes=2, target_label='high_erythrocytes')
# seed_everything(0)
# trainer = Trainer(max_epochs=1,
#                   precision=16,
#                   accumulate_grad_batches=2,
#                   enable_progress_bar=True,
#                   num_sanity_val_steps=0,
#                   callbacks=[lr_monitor, checkpoint_callback])
trainer = CurriculumTrainer()

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [ ]:
trainer.fit(model, train_data, 'erythrocytes', 'high_erythrocytes', 25, 0.01, 2, 0.34, 0.33, 0.33)

<ipython-input-12-1a796b59e397>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['distances'] = [dataframe[target_cells].mean() - value for value in dataframe[target_cells]]
<ipython-input-12-1a796b59e397>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['distances'] = (dataframe['distances'] - dataframe['distances'].mean()) / dataframe['distances'].std()
<ipython-input-12-1a796b59e397>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

-----------------------
Epoch 0, competence = 0.01


/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:565: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /content/drive/MyDrive/cells_dataset/checkpoints exists and is not empty.
INFO:pytorch_lightni

{'loss': tensor(0.7578, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0., device='cuda:0')}


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:369: `ModelCheckpoint(monitor='val_loss')` could not find the monitored key in the returned metrics: ['lr-AdamW', 'train_loss', 'train_metric', 'epoch', 'step']. HINT: Did you call `log('val_loss', value)` in the `LightningModule`?
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:565: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU availabl

...

{'loss': tensor(0.5148, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.7500, device='cuda:0')}
{'loss': tensor(0.5562, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.6250, device='cuda:0')}
{'loss': tensor(0.8799, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.5000, device='cuda:0')}
{'loss': tensor(0.3403, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.8750, device='cuda:0')}
{'loss': tensor(0.4329, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.8750, device='cuda:0')}
{'loss': tensor(0.4123, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.8750, device='cuda:0')}
{'loss': tensor(0.2061, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(1., device='cuda:0')}
{'loss': tensor(0.3258, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.8750, device='cuda:0')}
{'loss': tensor(1.1129, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.3750, devic

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:565: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)


-----------------------
Epoch 24, competence = 0.9591704749417592


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /content/drive/MyDrive/cells_dataset/checkpoints exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers`

{'loss': tensor(0.3845, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.7500, device='cuda:0')}
{'loss': tensor(0.3309, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(1., device='cuda:0')}
{'loss': tensor(0.5001, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.7500, device='cuda:0')}
{'loss': tensor(0.4156, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.8750, device='cuda:0')}
{'loss': tensor(0.5244, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.8750, device='cuda:0')}
{'loss': tensor(0.5111, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.7500, device='cuda:0')}
{'loss': tensor(0.3430, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(1., device='cuda:0')}
{'loss': tensor(0.7164, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.6250, device='cuda:0')}
{'loss': tensor(0.7696, device='cuda:0', grad_fn=<NllLossBackward0>), 'metric': tensor(0.6250, device='c

In [ ]:
trainer.validate(model)

In [ ]:
trainer.test(model)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir lightning_logs